In [2]:
# notebook specific things
from IPython.display import HTML, Markdown, display

display(HTML("<style>.container { width:90% !important; }</style>")) # Change report layout width

def markdown_print(string):
    display(Markdown(string))

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')



In [3]:
#-------------------------------------------------------------------------------
# Name:        RDFS Comparison
# Purpose:     Compare different versions of RDFS
#
# Author:      kristjan.vilgo
#
# Created:     01.02.2020
# Copyright:   (c) kristjan.vilgo 2020
# Licence:     GPL V2
#-------------------------------------------------------------------------------
import sys
sys.path.append("..")

import pandas

from RDF_parser import *
from RDFS_tools import *

# Set pandas display options
pandas.set_option('display.max_columns', 200)
pandas.set_option('display.max_rows', 350)
pandas.set_option('display.min_rows', 350)
pandas.set_option('display.expand_frame_repr', True)


In [4]:
# Load RDFS

path_old = r"..\rdfs\ENTSOE_CGMES_v2.4.15_04Jul2016_RDFS.zip"
path_new = r"..\rdfs\RDFS_UML_FDIS06_27Jan2020.zip"

data_new = pandas.read_RDF([path_new])
data_old = pandas.read_RDF([path_old])

data_set = {"new":data_new, "old": data_old}

WARNING 2 - Not supported file: RDFS_UML_FDIS06_27Jan2020/
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_DL.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_DY.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_EQBD.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_EQCO.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_GL.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_OP.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_SC.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_SSH.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_SV.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_TP.rdf
INFO - Loading RDFS_UML_FDIS06_27Jan2020/IEC61970-600-2_CDV_CGMES_3_0_0_TPBD.rdf
INFO - Loading EquipmentProfileCoreShortCircuitRDFSAugmented-v2_4

In [5]:
markdown_print("# Profiles")

profiles = {}
for data in data_set:
    profiles[data] = pandas.DataFrame(data_set[data]["INSTANCE_ID"].unique())

#for profiles
profiles["new"].merge(profiles["old"], how="outer", indicator="diff").replace({'left_only': r'added', 'right_only': 'removed'}).rename(columns={0:"Profile"})


# Profiles

,Profile,diff
0,#Package_DiagramLayoutProfile,both
1,#Package_DynamicsProfile,both
2,#Package_EquipmentBoundaryProfile,both
3,#Package_CoreEquipmentProfile,added
4,#Package_GeographicalLocationProfile,both
5,#Package_OperationProfile,added
6,#Package_ShortCircuitProfile,added
7,#Package_SteadyStateHypothesisProfile,both
8,#Package_StateVariablesProfile,both
9,#Package_TopologyProfile,both


In [6]:
report_name = "ConcreteClass"

markdown_print("# {}".format(report_name))

comparison = {}
for data in data_set:
    
    # Here goes query
    comparison[data] = pandas.DataFrame(data_set[data].query("VALUE=='http://iec.ch/TC57/NonStandard/UML#concrete'").ID.unique())

#for profiles
diff = comparison["new"].merge(comparison["old"], how="outer", indicator="diff").replace({'left_only': r'added', 'right_only': 'removed'}).rename(columns={0:report_name})

markdown_print("## Summary")
display(diff["diff"].value_counts())

markdown_print("## Removed")
removed = diff.query("diff=='removed'")
display(removed.merge(data_set["old"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())

markdown_print("## Added")
added = diff.query("diff=='added'")
display(added.merge(data_set["new"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())

# ConcreteClass

## Summary

both       318
added      44 
removed    9  
Name: diff, dtype: int64

## Removed

,ConcreteClass,diff,INSTANCE_ID
0,#DCEquipmentContainer,removed,#Package_EquipmentProfile
7,#DCEquipmentContainer,removed,#Package_TopologyProfile
33,#PerLengthDCLineParameter,removed,#Package_EquipmentProfile
53,#TapChangerTablePoint,removed,#Package_EquipmentProfile
73,#WindAeroLinearIEC,removed,#Package_DynamicsProfile
87,#WindGenTurbineType1IEC,removed,#Package_DynamicsProfile
101,#WindGenTurbineType3aIEC,removed,#Package_DynamicsProfile
115,#WindGenTurbineType3bIEC,removed,#Package_DynamicsProfile
129,#WindPitchContEmulIEC,removed,#Package_DynamicsProfile
143,http://entsoe.eu/CIM/SchemaExtension/3/1#EnergySchedulingType,removed,#Package_EquipmentBoundaryProfile


## Added

,ConcreteClass,diff,INSTANCE_ID
0,#CSCUserDefined,added,#Package_DynamicsProfile
7,#SVCUserDefined,added,#Package_DynamicsProfile
14,#VSCUserDefined,added,#Package_DynamicsProfile
21,#GovSteamBB,added,#Package_DynamicsProfile
28,#ExcNI,added,#Package_DynamicsProfile
35,#ExcRQB,added,#Package_DynamicsProfile
42,#PssRQB,added,#Package_DynamicsProfile
49,#PssSTAB2A,added,#Package_DynamicsProfile
56,#WindAeroOneDimIEC,added,#Package_DynamicsProfile
63,#WindAeroTwoDimIEC,added,#Package_DynamicsProfile


In [7]:
#data_set["new"].references_to("#ACLineSegment")
# TODO show all new classes

In [8]:
report_name = "UsedAssociations"

markdown_print("# {}".format(report_name))

properties = {}
for data in data_set:
    
    # Here goes query
    used_properties = data_set[data].query("KEY == 'AssociationUsed' & VALUE == 'Yes'").merge(data_set[data], on="ID", suffixes=("remove",""))[data_set[data].columns]
    properties[data] = pandas.DataFrame(used_properties.query("VALUE == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'").ID.unique())

#for profiles
diff = properties["new"].merge(properties["old"], how="outer", indicator="diff").replace({'left_only': r'added', 'right_only': 'removed'}).rename(columns={0:report_name})

markdown_print("## Summary")
display(diff["diff"].value_counts())

markdown_print("## Removed")
removed = diff.query("diff=='removed'")
display(removed.merge(data_set["old"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())

markdown_print("## Added")
added = diff.query("diff=='added'")
display(added.merge(data_set["new"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())


# UsedAssociations

## Summary

both       181
added      42 
removed    9  
Name: diff, dtype: int64

## Removed

,UsedAssociations,diff,INSTANCE_ID
0,#WindTurbineType3or4Dynamics.EnergySource,removed,#Package_DynamicsProfile
18,#WindGenTurbineType1IEC.WindAeroConstIEC,removed,#Package_DynamicsProfile
38,#WindGenTurbineType3IEC.WindAeroLinearIEC,removed,#Package_DynamicsProfile
58,#WindGenTurbineType3IEC.WindContPitchAngleIEC,removed,#Package_DynamicsProfile
78,#WindGenTurbineType3IEC.WindContPType3IEC,removed,#Package_DynamicsProfile
98,#WindGenTurbineType2IEC.WindPitchContEmulIEC,removed,#Package_DynamicsProfile
118,#WindGenTurbineType3IEC.WindMechIEC,removed,#Package_DynamicsProfile
138,http://entsoe.eu/CIM/SchemaExtension/3/1#EnergySource.EnergySchedulingType,removed,#Package_EquipmentBoundaryProfile
148,#DCLineSegment.PerLengthParameter,removed,#Package_EquipmentProfile


## Added

,UsedAssociations,diff,INSTANCE_ID
0,#WindTurbineType3or4Dynamics.PowerElectronicsConnection,added,#Package_DynamicsProfile
9,#StaticVarCompensatorDynamics.StaticVarCompensator,added,#Package_DynamicsProfile
18,#CSCDynamics.CSConverter,added,#Package_DynamicsProfile
27,#VSCDynamics.VsConverter,added,#Package_DynamicsProfile
36,#ProprietaryParameterDynamics.CSCUserDefined,added,#Package_DynamicsProfile
45,#ProprietaryParameterDynamics.SVCUserDefined,added,#Package_DynamicsProfile
54,#ProprietaryParameterDynamics.VSCUserDefined,added,#Package_DynamicsProfile
63,#CrossCompoundTurbineGovernorDynamics.HighPressureSynchronousMachineDynamics,added,#Package_DynamicsProfile
72,#CrossCompoundTurbineGovernorDynamics.LowPressureSynchronousMachineDynamics,added,#Package_DynamicsProfile
81,#WindGenTurbineType1aIEC.WindAeroConstIEC,added,#Package_DynamicsProfile


In [9]:
report_name = "Property"

markdown_print("# {}".format(report_name))

properties = {}
for data in data_set:
    
    # Here goes query
    properties[data] = pandas.DataFrame(data_set[data][~data_set[data].ID.str.contains("Version")].query("VALUE == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'").ID.unique())

#for profiles
diff = properties["new"].merge(properties["old"], how="outer", indicator="diff").replace({'left_only': r'added', 'right_only': 'removed'}).rename(columns={0:report_name})

markdown_print("## Summary")
display(diff["diff"].value_counts())

markdown_print("## Removed")
removed = diff.query("diff=='removed'")
display(removed.merge(data_set["old"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())

markdown_print("## Added")
added = diff.query("diff=='added'")
display(added.merge(data_set["new"], left_on=report_name, right_on="ID")[[report_name, "diff", "INSTANCE_ID"]].drop_duplicates())


# Property

## Summary

both       3374
added      323 
removed    152 
Name: diff, dtype: int64

## Removed

,Property,diff,INSTANCE_ID
0,#ActivePowerPerCurrentFlow.denominatorMultiplier,removed,#Package_EquipmentProfile
32,#ActivePowerPerCurrentFlow.denominatorUnit,removed,#Package_EquipmentProfile
64,#DCSeriesDevice.ratedUdc,removed,#Package_EquipmentProfile
96,#DCShunt.ratedUdc,removed,#Package_EquipmentProfile
128,#PerLengthDCLineParameter.capacitance,removed,#Package_EquipmentProfile
160,#CapacitancePerLength.value,removed,#Package_EquipmentProfile
188,#CapacitancePerLength.unit,removed,#Package_EquipmentProfile
220,#CapacitancePerLength.multiplier,removed,#Package_EquipmentProfile
252,#CapacitancePerLength.denominatorUnit,removed,#Package_EquipmentProfile
284,#CapacitancePerLength.denominatorMultiplier,removed,#Package_EquipmentProfile


## Added

,Property,diff,INSTANCE_ID
0,#WindTurbineType3or4Dynamics.PowerElectronicsConnection,added,#Package_DynamicsProfile
9,#PowerElectronicsConnection.WindTurbineType3or4Dynamics,added,#Package_DynamicsProfile
18,#StaticVarCompensatorDynamics.StaticVarCompensator,added,#Package_DynamicsProfile
27,#StaticVarCompensator.StaticVarCompensatorDynamics,added,#Package_DynamicsProfile
36,#CSCDynamics.CSConverter,added,#Package_DynamicsProfile
45,#CsConverter.CSCDynamics,added,#Package_DynamicsProfile
54,#VSCDynamics.VsConverter,added,#Package_DynamicsProfile
63,#VsConverter.VSCDynamics,added,#Package_DynamicsProfile
72,#CSCUserDefined.proprietary,added,#Package_DynamicsProfile
80,#CSCUserDefined.ProprietaryParameterDynamics,added,#Package_DynamicsProfile


In [48]:
report_name = "Multiplicity"

markdown_print("# {}".format(report_name))

properties = {}
for data in data_set:
    
    # Here goes query
    property_data = data_set[data].merge(data_set[data].query("VALUE == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'"), on="ID", suffixes=("", "_remove"))[data_set[data].columns].drop_duplicates()
    properties[data] = property_data.query("KEY=='multiplicity'")


common_property = properties["new"].merge(properties["old"], on=["ID","KEY", "INSTANCE_ID"], how="inner", suffixes=("_new", "_old"))
print("Common " + str(len(common_property)))


changes = common_property[~(common_property["VALUE_new"] == common_property["VALUE_old"])][["ID", "INSTANCE_ID", "VALUE_old", "VALUE_new"]]
print("Changes " + str(len(changes)))
display(changes)


# Multiplicity

Common 2910
Changes 59


,ID,INSTANCE_ID,VALUE_old,VALUE_new
31,#VisibilityLayer.VisibleObjects,#Package_DiagramLayoutProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..n,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..n
127,#RotatingMachineDynamics.damping,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
128,#RotatingMachineDynamics.inertia,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
131,#RotatingMachineDynamics.statorLeakageReactance,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
132,#RotatingMachineDynamics.statorResistance,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
195,#TurbineGovernorDynamics.SynchronousMachineDynamics,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..n,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1
201,#SynchronousMachineDetailed.efdBaseRatio,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
202,#SynchronousMachineDetailed.ifdBaseType,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
205,#SynchronousMachineTimeConstantReactance.ks,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1
206,#SynchronousMachineTimeConstantReactance.xDirectSync,#Package_DynamicsProfile,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1


In [45]:
report_name = "Comment"

markdown_print("# {}".format(report_name))

properties = {}
for data in data_set:
    
    # Here goes query
    property_data = data_set[data].merge(data_set[data].query("VALUE == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'"), on="ID", suffixes=("", "_remove"))[data_set[data].columns].drop_duplicates()
    properties[data] = property_data.query("KEY=='comment'")


common_property = properties["new"].merge(properties["old"], on=["ID","KEY", "INSTANCE_ID"], how="inner", suffixes=("_new", "_old"))
print("Common " + str(len(common_property)))


changes = common_property[~(common_property["VALUE_new"] == common_property["VALUE_old"])][["ID", "INSTANCE_ID", "VALUE_old", "VALUE_new"]]
print("Changes " + str(len(changes)))
display(changes)


# Comment

Common 2831
Changes 2509


,ID,INSTANCE_ID,VALUE_old,VALUE_new
0,#Diagram.orientation,#Package_DiagramLayoutProfile,Coordinate system orientation of the diagram.,"Coordinate system orientation of the diagram. A positive orientation gives standard “right-hand” orientation, with negative orientation indicating a “left-hand” orientation. For 2D diagrams, a positive orientation will result in X values increasing from left to right and Y values increasing from bottom to top. A negative orientation gives the “left-hand” orientation (favoured by computer graphics displays) with X values increasing from left to right and Y values increasing from top to bottom."
13,#DiagramObject.rotation,#Package_DiagramLayoutProfile,Sets the angle of rotation of the diagram object. Zero degrees is pointing to the top of the diagram. Rotation is clockwise.,"Sets the angle of rotation of the diagram object. Zero degrees is pointing to the top of the diagram. Rotation is clockwise. DiagramObject.rotation=0 has the following meaning: The connection point of an element which has one terminal is pointing to the top side of the diagram. The connection point ""From side"" of an element which has more than one terminal is pointing to the top side of the diagram.\nDiagramObject.rotation=90 has the following meaning: The connection point of an element which has one terminal is pointing to the right hand side of the diagram. The connection point ""From side"" of an element which has more than one terminal is pointing to the right hand side of the diagram."
14,#IdentifiedObject.DiagramObjects,#Package_DiagramLayoutProfile,The domain object to which this diagram object is associated.,The diagram objects that are associated with the domain object.
15,#DiagramObject.IdentifiedObject,#Package_DiagramLayoutProfile,The diagram objects that are associated with the domain object.,The domain object to which this diagram object is associated.
22,#DiagramObjectGluePoint.DiagramObjectPoints,#Package_DiagramLayoutProfile,The 'glue' point to which this point is associated.,A diagram object glue point is associated with 2 or more object points that are considered to be 'glued' together.
23,#DiagramObjectPoint.DiagramObjectGluePoint,#Package_DiagramLayoutProfile,A diagram object glue point is associated with 2 or more object points that are considered to be 'glued' together.,The 'glue' point to which this point is associated.
24,#DiagramObjectPoint.sequenceNumber,#Package_DiagramLayoutProfile,"The sequence position of the point, used for defining the order of points for diagram objects acting as a polyline or polygon with more than one point.","The sequence position of the point, used for defining the order of points for diagram objects acting as a polyline or polygon with more than one point. The attribute shall be a positive value."
28,#IdentifiedObject.mRID,#Package_DiagramLayoutProfile,"Master resource identifier issued by a model authority. The mRID is globally unique within an exchange context. Global uniqueness is easily achieved by using a UUID, as specified in RFC 4122, for the mRID. The use of UUID is strongly recommended.\nFor CIMXML data files in RDF syntax conforming to IEC 61970-552 Edition 1, the mRID is mapped to rdf:ID or rdf:about attributes that identify CIM object elements.","Master resource identifier issued by a model authority. The mRID is unique within an exchange context. Global uniqueness is easily achieved by using a UUID, as specified in RFC 4122, for the mRID. The use of UUID is strongly recommended.\nFor CIMXML data files in RDF syntax conforming to IEC 61970-552, the mRID is mapped to rdf:ID or rdf:about attributes that identify CIM object elements."
29,#IdentifiedObject.mRID,#Package_DynamicsProfile,"Master resource identifier issued by a model authority. The mRID is globally unique within an exchange context. Global uniqueness is easily achieved by using a UUID, as specified in RFC 4122, for the mRID. The use of UUID is strongly recommended.\nFor CIMXML data files in RDF s

In [49]:
data_set["new"].query("ID=='#RotatingMachineDynamics.damping'")

,ID,KEY,VALUE,INSTANCE_ID
3253,#RotatingMachineDynamics.damping,Type,Description,#Package_DynamicsProfile
3254,#RotatingMachineDynamics.damping,stereotype,http://iec.ch/TC57/NonStandard/UML#attribute,#Package_DynamicsProfile
3255,#RotatingMachineDynamics.damping,label,damping,#Package_DynamicsProfile
3256,#RotatingMachineDynamics.damping,domain,#RotatingMachineDynamics,#Package_DynamicsProfile
3257,#RotatingMachineDynamics.damping,dataType,#Float,#Package_DynamicsProfile
3258,#RotatingMachineDynamics.damping,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1,#Package_DynamicsProfile
3259,#RotatingMachineDynamics.damping,comment,Damping torque coefficient (,#Package_DynamicsProfile
3260,#RotatingMachineDynamics.damping,type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property,#Package_DynamicsProfile


In [44]:
property_data

,ID,KEY,VALUE,INSTANCE_ID
0,#EquipmentVersion.baseUML,Type,Description,#Package_EquipmentProfile
4,#EquipmentVersion.baseUML,stereotype,http://iec.ch/TC57/NonStandard/UML#attribute,#Package_EquipmentProfile
8,#EquipmentVersion.baseUML,label,baseUML,#Package_EquipmentProfile
12,#EquipmentVersion.baseUML,domain,#EquipmentVersion,#Package_EquipmentProfile
16,#EquipmentVersion.baseUML,dataType,#String,#Package_EquipmentProfile
20,#EquipmentVersion.baseUML,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:1..1,#Package_EquipmentProfile
24,#EquipmentVersion.baseUML,isFixed,iec61970cim16v28_iec61968cim12v08_iec62325cim03v01a,#Package_EquipmentProfile
28,#EquipmentVersion.baseUML,comment,Base UML provided by CIM model manager.,#Package_EquipmentProfile
32,#EquipmentVersion.baseUML,type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property,#Package_EquipmentProfile
144,#EquipmentVersion.baseURIcore,Type,Description,#Package_EquipmentProfile


In [19]:
property_data = data_set[data].merge(data_set[data][~data_set[data].ID.str.contains("Version")].query("VALUE == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'"), on="ID", suffixes=("", "_remove"))[data_set[data].columns].drop_duplicates()



In [21]:
property_data.query("KEY=='multiplicity'")

,ID,KEY,VALUE,INSTANCE_ID
20,#ACDCConverter.baseS,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
158,#ApparentPower.value,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
200,#ApparentPower.value,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_DynamicsProfile
410,#ApparentPower.unit,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
458,#ApparentPower.unit,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_DynamicsProfile
698,#ApparentPower.multiplier,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
746,#ApparentPower.multiplier,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_DynamicsProfile
976,#ACDCConverter.idleLoss,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
1124,#ActivePower.value,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_EquipmentProfile
1180,#ActivePower.value,multiplicity,http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#M:0..1,#Package_StateVariablesProfile


In [ ]:
#property_data.pivot(index="ID", columns="KEY", values="VALUE")

In [ ]:
data_set[data].query("KEY == 'AssociationUsed' & VALUE == 'Yes'").merge(data_set[data], on="ID", suffixes=("remove",""))[data_set[data].columns]

In [ ]:
topology = data_new.query("INSTANCE_ID == '#Package_TopologyBoundaryProfile'")

pandas.set_option("display.max_rows", 40)
pandas.set_option("display.max_columns", 20)

ClassCategory = 'http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#ClassCategory'
Class = 'http://www.w3.org/2000/01/rdf-schema#Class'
Property = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'


In [ ]:
topology.query("VALUE == '{}'".format(Class))

In [ ]:
topology.query("ID=='#String'")

In [ ]:
topology.type_tableview("Description").query("type == '{}'".format(Class)).dropna(how="all", axis="columns")

In [ ]:
topology.type_tableview("Description").query("type == '{}'".format(Class)).columns

In [ ]:
validation_view(topology, "#TopologyBoundaryProfileVersion")

In [ ]:
for ID in topology.query("VALUE == '{}'".format(Class)).ID:
    print(ID)
    display(parameter_tableview(topology, ID))